#Install

In [ ]:
#@title download font
!wget https://github.com/rastikerdar/vazir-font/releases/download/v30.1.0/Vazir-font-v30.1.0.zip
!unzip Vazir-font-v30.1.0.zip -d /usr/share/fonts/truetype/

In [ ]:
#@title نصب کتابخانه‌های مورد نیاز
!pip install PyPDF2
!pip install google-generativeai
!pip install tqdm
!pip install reportlab
!pip install arabic-reshaper

# Settings

In [ ]:

#@title فراخوانی کلید دسترسی
import google.generativeai as genai

GOOGLE_API_KEY = "" #@param {type:"string"}
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
#@title آپلود و تنظیم صفحات
from google.colab import files
import PyPDF2
import io

# دریافت ورودی صفحات از کاربر
start_page = 1  #@param {type:"slider", min:1, max:1000, step:1}
end_page = 1  #@param {type:"slider", min:1, max:1000, step:1}

# آپلود فایل
uploaded = files.upload()

if not uploaded:
    raise ValueError("هیچ فایلی آپلود نشد. لطفاً یک فایل PDF آپلود کنید.")

filename = list(uploaded.keys())[0]

# بررسی فرمت فایل
if not filename.lower().endswith('.pdf'):
    raise ValueError("فرمت فایل اشتباه است. لطفاً یک فایل PDF آپلود کنید.")

# خواندن فایل PDF
pdf_reader = PyPDF2.PdfReader(io.BytesIO(uploaded[filename]))

# بررسی محدوده صفحات
num_pages = len(pdf_reader.pages)
if start_page < 1 or end_page > num_pages:
    raise ValueError(f"شماره صفحات باید بین 1 و {num_pages} باشد.")

#Translate

In [ ]:


#@title ترجمه به فارسی
from tqdm.notebook import tqdm
import time
from tenacity import retry, stop_after_attempt, wait_exponential

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def translate_page_with_retry(text):
    model = genai.GenerativeModel('gemini-1.5-flash')

    prompt = f"""متن زیر را به فارسی عامیانه و روان ترجمه کن. نکات مهم:

1. فقط ترجمه خالص و عامیانه، بدون هیچ توضیح اضافه
2. به شدت مهمه که تا معنای واقعی جملات رو درک نکنی حق ترجمه نداری
3. استفاده از زبان محاوره‌ای و روزمره
4. ترجمه نباید بی ربط باشه و سعی کن تا جایی که ممکنه حرفه ای عمل کنی
5. حفظ پاراگراف بندی متن


متن برای ترجمه:
{text}"""

    response = model.generate_content(prompt)
    time.sleep(2)
    return response.text

# شروع ترجمه صفحات انتخاب شده
translated_text = ""
with tqdm(total=end_page-start_page+1, desc="ترجمه صفحات") as pbar:
    for page_num in range(start_page-1, end_page):
        try:
            page_text = pdf_reader.pages[page_num].extract_text()
            translated_page = translate_page_with_retry(page_text)
            translated_text += f"\n=== صفحه {page_num+1} ===\n{translated_page}\n"
            pbar.update(1)

            # ذخیره موقت
            translated_text_encoded = translated_text.encode('utf-8-sig').decode('utf-8-sig')
            with open("translated_backup.txt", "w", encoding="utf-8-sig") as f:
                f.write(translated_text_encoded)

        except Exception as e:
            print(f"\nخطا در صفحه {page_num+1}")
            raise

# ذخیره نهایی
translated_text_encoded = translated_text.encode('utf-8-sig').decode('utf-8-sig')
with open(f"{filename}_translated.txt", "w", encoding="utf-8-sig") as f:
    f.write(translated_text_encoded)

print("\nترجمه با موفقیت به پایان رسید!")

In [ ]:
#@title دانلود
from google.colab import files
files.download(f"{filename}_translated.txt")